#### Import requirements, define global variables

In [3]:
# Import the required libraries
from arcgis.gis import GIS
import json
import requests
from datetime import datetime

In [4]:
# Get username and password from config file 
with open('config.json') as config_file:
    config = json.load(config_file)

AGO_username = config['AGO_username']
AGO_password = config['AGO_password']
webeoc_username = config['webeoc_username']
webeoc_password = config['webeoc_password']

# Get the current date-time
current_datetime = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

In [5]:
# Sign into the ArcGIS Online or Enterprise Portal
gis = GIS("https://ghis.maps.arcgis.com/", AGO_username, AGO_password)

In [15]:
# Get the feature layer item
feature_service = gis.content.get('4cec9a93384543e0a676e3ad892362bb')

# Access the feature layer
incidentTbl = feature_service.tables[2]

# Convert the incident table to a dataframe
incident_df = incidentTbl.query(as_df=True)
incident_df.head()

,OBJECTID,GlobalID,HazardID,Name,Description,StartDate,EndDate,IsDeleted,Creator,CreationDate,Editor,EditDate
0,1,e8db1093-90e7-4503-bd69-ab2118bd4073,d349d7b3-f251-41f7-a899-2f5dfffc88ef,Virginia Wildfire August 2024,August Wildfire,2024-08-07 00:00:00,NaT,<NA>,AadhikariGH,2024-08-07 17:35:34.809000015,AadhikariGH,2024-08-07 17:35:34.809000015
1,2,7ceb81f3-8c33-40b1-9150-c1f2b8bad868,d349d7b3-f251-41f7-a899-2f5dfffc88ef,Market Fire,test,2024-10-17 04:00:00,NaT,0,bntiamoahGH,2024-10-17 11:35:47.996999979,bntiamoahGH,2024-10-17 11:35:47.996999979
2,3,351d7d86-dfbc-468c-bb86-a4c8648f6fef,ed102c3e-1c36-461b-86cd-57b7ca9bba85,Locust Infestation,test,2024-10-17 04:00:00,NaT,0,bntiamoahGH,2024-10-17 11:37:15.862999916,bntiamoahGH,2024-10-17 11:37:15.862999916


In [35]:
incidentRecord = json.loads(incident_df.head(1).to_json(orient='records'))
name = incidentRecord[0]['Name']
description = incidentRecord[0]['Description']
creationDate = incidentRecord[0]['StartDate']

In [45]:
# Create the session to be utilized by the REST API.
base_url = 'https://trial1.demo.webeocasp.com/trial1/api/rest.svc'

# Set Default Incident and Position
position = "CMD Incident Commander"
incident = "Trial"

def create_session(base_url):
	url = f"{base_url}/sessions"

	payload = json.dumps({
		"username": webeoc_username,
		"password": webeoc_password,
		"position": position,
		"incident": incident
	})
	headers = {
		'Content-Type': 'application/json',
	}
	session = requests.Session()
	session.post(url, headers=headers, data=payload)

	return session

# End the session.
def close_session(session):
  url = f"{base_url}/sessions"
  session.delete(url)

In [46]:
# Create a POST Request to create a record using the provided details.
def create_board_record(base_url, session, board_name, inputviewname, record_data):
	create_record_url = f"{base_url}/board/{board_name}/input/{inputviewname}"
	print (create_record_url)
	payload = {'data': json.dumps(record_data)}

	response = session.post(create_record_url, json=payload)

	if response.status_code == 201:
		# Record created successfully, return the response
		return response.json()
	else:
		# Record creation failed, raise an exception with the error message
		response.raise_for_status()

In [47]:
#Map data from CLSS Incident to WebEOC Board Record

###### Example usage - Event Reporting: Sig Event
board_name = 'Incident Creator'
view_name = 'Incident Input'
# ['dataid', 'incidentid', 'userid', 'positionid', 'entrydate', 'subscribername', 'prevdataid', 'globalid', 'review_number', 'incident_activationSummary', 'incident_agency', 'incident_comments', 'incident_location', 'incident_point_of_contact', 'incident_contact_number', 'incident_isActive', 'webeoc_incident_name', 'incident_creationDate', 'incident_eventType', 'incident_eventTypeOther', 'incident_activationLevel', 'incident_activationType', 'incident_createdBy_user', 'incident_createdBy_position', 'location']

record_data = {
	'webeoc_incident_name': name,
    'incident_comments': description,
    'incident_creationDate': creationDate,
    'incident_createdBy_user': webeoc_username,
    'incident_createdBy_position': position
}

def main():
	
	session = create_session(base_url)

	try:
		response = create_board_record(base_url, session, board_name, view_name, record_data)
		print("Record created successfully:", response)
	except requests.exceptions.RequestException as e:
		print("Failed to create record:", e)

	close_session(session)

main()

https://trial1.demo.webeocasp.com/trial1/api/rest.svc/board/Incident Creator/input/Incident Input
Record created successfully: None
